tmp.ipynb 에서 이어짐.

- 강수량 / mm
- 강수형태 / 비옴 : 1, 비안옴 : 0
- 강수확률 / % step : 10
- 기온 / 'C step : 1
- 습도 / % step : 5
- 풍속 / ms step 0.1
- 하늘 상태 / 맑음(1), 구름많음(3), 흐림(4)

- 강수량 / mm step : 1 -> 그냥 int
- 강수형태 / 비옴 : 1, 비안옴 : 0 -> 그냥 int
- 강수확률 / % step : 10 -> 그냥 int
- 기온 / 'C step : 1 -> 그냥 int
- 습도 / % step : 5 -> 그냥 int
- 풍속 / ms step 0.1 -> X 10 하고 int
- 하늘 상태 / 맑음(1), 구름많음(3), 흐림(4) -> 그냥 int

In [4]:
import os
import numpy as np

In [5]:
def loadData(filePath : str, convertFunction = lambda x : int(float(x)), removeDate : bool = True):
	rawData = []
	with open(filePath) as f:
		rawData = list(map(lambda x : x.rstrip().lstrip().split(','), f.readlines()))

	data = []
	lastStart = 1
	for i, objs in enumerate(rawData[:-1]):
		if objs[0][:5] == "Start":
			if removeDate and (objs[0][8:] in ['20220117', '20220203', '20220204']):
				lastStart = i+1
			else:
				data.append(list(map(lambda x : convertFunction(x[3]), rawData[lastStart:i])))
				lastStart = i+1
	del data[0]
	# 0번째 데이터는 개수가 안 맞음

	return np.array(data, dtype=np.int8)

In [6]:
filePaths = {
	fileName.split('.')[0] : f"datasets/{fileName}" for fileName in os.listdir('datasets')
}

In [18]:
rawDataset = {}
for key, value in filePaths.items():
	if key == '풍속':
		func = lambda x : int(float(x) * 10)
	else:
		func = lambda x : int(float(x))

	data = loadData(value, func, False if key == '기온' else True)
	rawDataset[key] = data
	print(f"{key} {data.shape}")

강수량 (1123, 508)
강수형태 (1123, 508)
강수확률 (1123, 508)
기온 (1123, 508)
습도 (1123, 508)
풍속 (1123, 508)
하늘상태 (1123, 508)


In [19]:
rawDataset['강수량']

array([[2, 0, 0, ..., 4, 6, 6],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

전처리

- 강수량 / 0 ~ 50 -> 0 ~ 1. => 얘를 쓸지 말지는 고민 일단은 안 씀
- 강수형태 / 1, 0 => 얘가 DataY
- 강수확률 / 0 ~ 1. => 얘도 고민 중 일단은 안 씀
- 기온 / -20 ~ 40 -> 0 ~ 1.
- 습도 / 0 ~ 100 -> 0 ~ 1.
- 풍속 / 0 ~ 200(20m/s) -> 0 ~ 1.
- 하늘 상태 / 맑음(1), 구름많음(3), 흐림(4) -> 맑 : 0, 구름 많, 흐림 : 1

### DataX
- 기온 (-20 ~ 40 => 0 ~ 1.)
- 습도 (0 ~ 100 => 0 ~ 1.)
- 풍속 (0 ~ 200(20m/s) => 0 ~ 1.)
- 하늘상태 (맑 : 0, 구름많 & 흐 : 1)

### DataY
- 강수형태

In [44]:
max(rawDataset['기온'].flatten()), min(rawDataset['기온'].flatten())

(35, -16)

In [20]:
max(rawDataset['강수량'].flatten())

50

In [26]:
max(rawDataset['풍속'].flatten())

127

In [46]:
dataset = {}

dataset['습도'] = rawDataset['습도'].flatten()/100
dataset['풍속'] = rawDataset['풍속'].flatten()/200
dataset['하늘상태'] = np.where(rawDataset['하늘상태'].flatten() == 1, 0, 1)
dataset['기온'] = (rawDataset['기온'].flatten() + 20)/60